In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/legal-queries/allQuestions.csv


In [2]:
pip install -q transformers datasets accelerate scikit-learn numpy pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 10.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 45.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

In [17]:
import os, json, random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup, DataCollatorWithPadding
from torch.optim import AdamW

## COMBINING CV SETS

In [18]:
csv_path = "/kaggle/input/legal-queries/allQuestions.csv"  
text_col = "question"
label_col = "act"
model_names = ["roberta-base", "distilroberta-base", "nlpaueb/legal-bert-base-uncased"]  # tries in order
max_len = 256
test_size = 0.2
seed = 42
epochs = 4
batch_size = 16
lr = 2e-5
weight_decay = 0.01
warmup_ratio = 0.1
out_dir = "/kaggle/working/robertaActsCls"

In [19]:
# ====== SEED & DEVICE ======
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ====== LOAD DATA ======
df = pd.read_csv(csv_path, encoding="utf-8-sig").dropna(subset=[text_col, label_col])
df[text_col] = df[text_col].astype(str).str.strip()
df[label_col] = df[label_col].astype(str).str.strip()

# ====== ENCODE LABELS & SPLIT ======
le = LabelEncoder()
df["label_id"] = le.fit_transform(df[label_col])
id2label = {i: lab for i, lab in enumerate(le.classes_)}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

In [20]:
train_df, test_df = train_test_split(
    df, test_size=test_size, random_state=seed, stratify=df["label_id"]
)

print(f"Train {len(train_df)} | Test {len(test_df)} | Classes {num_labels}")


Train 3727 | Test 932 | Classes 20


In [21]:
tok = None; model = None; chosen_model = None
last_err = None
for name in model_names:
    try:
        tok = AutoTokenizer.from_pretrained(name, use_fast=True)
        model = AutoModelForSequenceClassification.from_pretrained(
            name, num_labels=num_labels, label2id=label2id, id2label=id2label
        )
        chosen_model = name
        break
    except Exception as e:
        last_err = e
if model is None:
    raise RuntimeError(f"Could not load any model from {model_names}. Last error: {last_err}")

print("Using model:", chosen_model)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using model: roberta-base


In [22]:
class TxtCls(Dataset):
    def __init__(self, texts, labels):
        self.texts = list(texts)
        self.labels = list(labels)
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, i):
        return {"text": self.texts[i], "labels": int(self.labels[i])}

collator = DataCollatorWithPadding(tokenizer=tok)

def collate_fn(batch):
    texts = [b["text"] for b in batch]
    labels = torch.tensor([b["labels"] for b in batch], dtype=torch.long)
    enc = tok(texts, truncation=True, max_length=max_len, padding=True, return_tensors="pt")
    enc["labels"] = labels
    return enc


In [23]:
train_ds = TxtCls(train_df[text_col].values, train_df["label_id"].values)
test_ds  = TxtCls(test_df[text_col].values,  test_df["label_id"].values)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# ====== OPTIM, SCHED ======
model.to(device)
optim = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
total_steps = len(train_loader) * epochs
warmup_steps = int(total_steps * warmup_ratio)
sched = get_linear_schedule_with_warmup(optim, num_warmup_steps=warmup_steps, num_training_steps=total_steps)


In [24]:
def evaluate():
    model.eval()
    all_true, all_pred = [], []
    loss_sum, n = 0.0, 0
    with torch.no_grad():
        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            out = model(**batch)
            loss = out.loss
            logits = out.logits
            preds = logits.argmax(dim=-1).detach().cpu().numpy().tolist()
            true  = batch["labels"].detach().cpu().numpy().tolist()
            all_true += true
            all_pred += preds
            loss_sum += float(loss.item()) * len(true)
            n += len(true)
    acc = accuracy_score(all_true, all_pred)
    f1_macro = f1_score(all_true, all_pred, average="macro", zero_division=0)
    f1_weighted = f1_score(all_true, all_pred, average="weighted", zero_division=0)
    return {"loss": loss_sum / max(n,1), "accuracy": acc, "f1_macro": f1_macro, "f1_weighted": f1_weighted}, all_true, all_pred


In [26]:
best_f1 = -1.0
for ep in range(1, epochs + 1):
    model.train()
    running = 0.0
    for step, batch in enumerate(train_loader, 1):
        batch = {k: v.to(device) for k, v in batch.items()}
        out = model(**batch)
        loss = out.loss
        loss.backward()
        optim.step()
        sched.step()
        optim.zero_grad()
        running += float(loss.item())
        if step % 100 == 0:
            print(f"epoch {ep} step {step}/{len(train_loader)} train_loss {running/step:.4f}")

    train_loss = running / max(1, len(train_loader))
    metrics, y_true, y_pred = evaluate()
    val_loss = metrics["loss"]
    print(f"epoch {ep} | train_loss {train_loss:.4f} | val_loss {val_loss:.4f} | acc {metrics['accuracy']:.4f} | f1_macro {metrics['f1_macro']:.4f} | f1_weighted {metrics['f1_weighted']:.4f}")

    if metrics["f1_macro"] > best_f1:
        best_f1 = metrics["f1_macro"]
        os.makedirs(out_dir, exist_ok=True)
        model.save_pretrained(out_dir)
        tok.save_pretrained(out_dir)
        with open(os.path.join(out_dir, "id2label.json"), "w", encoding="utf-8") as f:
            json.dump(id2label, f, ensure_ascii=False, indent=2)
        print("saved best to", out_dir)



epoch 1 step 100/233 train_loss 1.2364
epoch 1 step 200/233 train_loss 1.1880
epoch 1 | train_loss 1.1610 | val_loss 0.9810 | acc 0.7339 | f1_macro 0.6266 | f1_weighted 0.7254
saved best to /kaggle/working/robertaActsCls
epoch 2 step 100/233 train_loss 0.8325
epoch 2 step 200/233 train_loss 0.8174
epoch 2 | train_loss 0.8129 | val_loss 0.9053 | acc 0.7543 | f1_macro 0.6441 | f1_weighted 0.7496
saved best to /kaggle/working/robertaActsCls
epoch 3 step 100/233 train_loss 0.6585
epoch 3 step 200/233 train_loss 0.6275
epoch 3 | train_loss 0.6271 | val_loss 0.8708 | acc 0.7650 | f1_macro 0.6509 | f1_weighted 0.7593
saved best to /kaggle/working/robertaActsCls
epoch 4 step 100/233 train_loss 0.5601
epoch 4 step 200/233 train_loss 0.5611
epoch 4 | train_loss 0.5576 | val_loss 0.8693 | acc 0.7672 | f1_macro 0.6562 | f1_weighted 0.7620
saved best to /kaggle/working/robertaActsCls


In [27]:

metrics, y_true, y_pred = evaluate()
print("\nOverall:", metrics)
print("\nReport:\n", classification_report(
    y_true, y_pred, target_names=[id2label[i] for i in range(num_labels)], digits=4, zero_division=0
))


Overall: {'loss': 0.8692839329846427, 'accuracy': 0.7671673819742489, 'f1_macro': 0.6561756629924154, 'f1_weighted': 0.7619751849874999}

Report:
                                precision    recall  f1-score   support

                 Children Act     0.7458    0.8381    0.7892       105
        Constitution of Kenya     0.8529    0.8878    0.8700        98
      Criminal Procedure Code     0.8222    0.8952    0.8571       124
          Data Protection Act     0.9444    0.7391    0.8293        23
        Distress for Rent Act     0.8750    0.7000    0.7778        10
               Employment Act     0.6176    0.5676    0.5915        37
                 Evidence Act     0.7111    0.6275    0.6667        51
              Excise Duty Act     0.6923    0.6429    0.6667        28
          Fatal Accidents Act     0.0000    0.0000    0.0000         3
               Income Tax Act     0.7350    0.8269    0.7783       104
         Labour Relations Act     0.7742    0.8000    0.7869        30

In [28]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_recall_fscore_support

# class-weighted CE
counts = train_df["label_id"].value_counts().sort_index().values
weights = (len(train_df) / (len(counts) * counts))
weights = torch.tensor(weights, dtype=torch.float).to(device)
criterion = torch.nn.CrossEntropyLoss(weight=weights)

In [29]:

def evaluate():
    model.eval()
    all_true, all_pred = [], []
    loss_sum, n = 0.0, 0
    with torch.no_grad():
        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            out = model(**{k: batch[k] for k in ["input_ids","attention_mask"]})
            logits = out.logits
            loss = criterion(logits, batch["labels"])
            preds = logits.argmax(dim=-1).detach().cpu().numpy().tolist()
            true  = batch["labels"].detach().cpu().numpy().tolist()
            all_true += true
            all_pred += preds
            loss_sum += float(loss.item()) * len(true)
            n += len(true)
    acc = accuracy_score(all_true, all_pred)
    f1_macro = f1_score(all_true, all_pred, average="macro", zero_division=0)
    f1_weighted = f1_score(all_true, all_pred, average="weighted", zero_division=0)
    return {"loss": loss_sum / max(n,1), "accuracy": acc, "f1_macro": f1_macro, "f1_weighted": f1_weighted}, all_true, all_pred

best_f1 = -1.0
bad_epochs = 0
patience = 2
out_dir1 ="/kaggle/working/roberaActsCls"

In [ ]:
for ep in range(1, epochs + 1):
    model.train()
    running = 0.0
    for step, batch in enumerate(train_loader, 1):
        batch = {k: v.to(device) for k, v in batch.items()}
        out = model(**{k: batch[k] for k in ["input_ids","attention_mask"]})
        logits = out.logits
        loss = criterion(logits, batch["labels"])
        loss.backward()
        optim.step()
        sched.step()
        optim.zero_grad()
        running += float(loss.item())
        if step % 100 == 0:
            print(f"epoch {ep} step {step}/{len(train_loader)} train_loss {running/step:.4f}")

    train_loss = running / max(1, len(train_loader))
    metrics, y_true, y_pred = evaluate()
    val_loss = metrics["loss"]
    print(f"epoch {ep} | train_loss {train_loss:.4f} | val_loss {val_loss:.4f} | acc {metrics['accuracy']:.4f} | f1_macro {metrics['f1_macro']:.4f} | f1_weighted {metrics['f1_weighted']:.4f}")

    if metrics["f1_macro"] > best_f1 + 1e-4:
        best_f1 = metrics["f1_macro"]
        bad_epochs = 0
        os.makedirs(out_dir, exist_ok=True)
        model.save_pretrained(out_dir)
        tok.save_pretrained(out_dir)
        with open(os.path.join(out_dir, "id2label.json"), "w", encoding="utf-8") as f:
            json.dump(id2label, f, ensure_ascii=False, indent=2)
        print("saved best to", out_dir)
    else:
        bad_epochs += 1
        if bad_epochs >= patience:
            print("early stop")
            break



epoch 1 step 100/233 train_loss 0.7590
epoch 1 step 200/233 train_loss 0.8463
epoch 1 | train_loss 0.8523 | val_loss 1.2549 | acc 0.7672 | f1_macro 0.6562 | f1_weighted 0.7620
saved best to /kaggle/working/robertaActsCls
epoch 2 step 100/233 train_loss 0.8322
epoch 2 step 200/233 train_loss 0.8330
epoch 2 | train_loss 0.8443 | val_loss 1.2549 | acc 0.7672 | f1_macro 0.6562 | f1_weighted 0.7620
epoch 3 step 100/233 train_loss 0.9158
epoch 3 step 200/233 train_loss 0.8593


In [1]:
metrics, y_true, y_pred = evaluate()
print("\nOverall:", metrics)
print("\nReport:\n", classification_report(
    y_true, y_pred, target_names=[id2label[i] for i in range(num_labels)], digits=4, zero_division=0
))

NameError: name 'evaluate' is not defined